In [21]:
from __init__ import * 

import snmcseq_utils
from snmcseq_utils import create_logger
from snmcseq_utils import get_cluster_mc_c
# from CEMBA_update_mysql import connect_sql
# from CEMBA_init_ensemble_v2 import pull_genebody_info
# from CEMBA_init_ensemble_v2 import pull_binc_info

In [8]:
log = create_logger()

In [9]:
# ens = 'Ens0'
# context = 'CH'
# df = get_cluster_mc_c(ens, context, genome_regions='bin', cluster_type='', database='human_snmcseq')

In [10]:
f = '/cndd/Public_Datasets/human_snmcseq/References/Human_reference/binc_mCH_human_combined_100000_summary.tsv' 
df_input = pd.read_table(f, index_col=['chr', 'bin'], dtype={'chr': object})
df_c = df_input.filter(regex='_c$')
df_mc = df_input.filter(regex='_mc$')



In [12]:
print(df_input.shape)

(30353, 12870)


In [16]:
f = '/cndd/Public_Datasets/human_snmcseq/References/Human_reference/Ens0.tsv'
df_meta = pd.read_table(f) #, index_col='cell_name')
print(df_meta.shape)
df_meta.head()

(6435, 4)


,cell_name,dataset,cluster,annotation
0,160729_MB_v1_hs_25yr_MFG_pool_1000_AD008_indexed,MB_v1,8,hSst-2
1,160729_MB_v1_hs_25yr_MFG_pool_1019_AD006_indexed,MB_v1,8,hSst-2
2,160729_MB_v1_hs_25yr_MFG_pool_101_AD010_indexed,MB_v1,8,hSst-2
3,160729_MB_v1_hs_25yr_MFG_pool_1022_AD006_indexed,MB_v1,8,hSst-2
4,160729_MB_v1_hs_25yr_MFG_pool_1033_AD002_indexed,MB_v1,8,hSst-2


In [19]:
df_mc_c = pd.DataFrame() 
for label, df_sub in df_meta.groupby('annotation'):
    samples = df_sub['cell_name'].values
    df_mc_c['{}_mc'.format(label)] = df_mc[samples+'_mc'].sum(axis=1)
    df_mc_c['{}_c'.format(label)] = df_c[samples+'_c'].sum(axis=1)

In [22]:
df_mcc = snmcseq_utils.get_mcc(df_mc_c, base_call_cutoff=100)

09/10/2018 07:07:18 PM Getting mcc matrix from mc and c
09/10/2018 07:07:18 PM base_call_cutoff=100, sufficient_coverage_fraction=1
09/10/2018 07:07:18 PM Matrix size before pruning... (30353, 40)
09/10/2018 07:07:18 PM Matrix size after pruning... (28473, 40)
09/10/2018 07:07:18 PM (28473, 20)
09/10/2018 07:07:18 PM Imputing data... (No effect if sufficient_coverage_fraction=1)


In [23]:
print(df_mcc.shape)
df_mcc.head()

(28473, 20)


hDL-1_mcc  hDL-2_mcc  hGlia_mcc  hL2/3_mcc   hL4_mcc  hL5-1_mcc  \
chr bin                                                                       
1   0        0.040500   0.042511   0.007041   0.034970  0.025196   0.039137   
    100000   0.066646   0.072746   0.010790   0.059102  0.047823   0.061425   
    200000   0.050578   0.053612   0.009415   0.041798  0.039564   0.058382   
    400000   0.066056   0.081181   0.004847   0.070736  0.072100   0.042918   
    500000   0.050793   0.058928   0.010903   0.047387  0.037621   0.052144   

            hL5-2_mcc  hL5-3_mcc  hL5-4_mcc  hL6-1_mcc  hL6-2_mcc  hL6-3_mcc  \
chr bin                                                                        
1   0        0.020749   0.066215   0.044694   0.036644   0.042989   0.027425   
    100000   0.053498   0.094434   0.078606   0.061858   0.076596   0.050719   
    200000   0.036982   0.087800   0.056535   0.045757   0.049344   0.036055   
    400000   0.040650   0.101818   0.060353   0.045685   0.050360   0.054591   
    500000   0.034461   0.085825   0.061929   0.063268   0.064824   0.037560   

            hNdnf_mcc  hNos_mcc  hPv-1_mcc  hPv-2_mcc  hSst-2_mcc  hSst-3_mcc  \
chr bin                                                                         
1   0        0.018572  0.025421   0.026845   0.026412    0.023155    0.043746   
    100000   0.037164  0.049270   0.057413   0.053653    0.042189    0.075603   
    200000   0.028414  0.039037   0.043750   0.045278    0.037899    0.059921   
    400000   0.051576  0.068293   0.051804   0.044534    0.041518    0.058419   
    500000   0.031556  0.033847   0.033966   0.035246    0.026700    0.046232   

            hVip-1_mcc  hVip-2_mcc  
chr bin                             
1   0         0.015219    0.020133  
    100000    0.027410    0.046906  
    200000    0.022813    0.035847  
    400000    0.121693    0.075540  
    500000    0.031556    0.037087

In [24]:
fout = '/cndd/Public_Datasets/human_snmcseq/References/Human_reference/binc_mCH_human_combined_100000_clusterwise_mcc.tsv'
df_mcc.to_csv(fout, sep="\t", na_rep='NA', index=True, header=True)